In [2]:

!git clone https://github.com/miguel-fc/Neutron_Reflect.git

fatal: destination path 'Neutron_Reflect' already exists and is not an empty directory.


In [3]:
cd Neutron_Reflect/

/pscratch/sd/q/qubri000/Neutron_Reflect


In [4]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gdown

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -U --no-cache-dir gdown --pre

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [9]:
#import files

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import numpy as np
import data_preparation as dpre
import io
import os
# import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
import seaborn as sea

In [10]:
#CNN structure

class CNN(nn.Module):
    def __init__(self, num_layers):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        addition = 255/num_layers
        curr = 1
        for hdim in range(num_layers-1):
            self.layers.append(nn.Conv1d(int(curr+0.5), int(curr+addition+0.5),51,padding=25))
            self.layers.append(nn.BatchNorm1d(int(curr+addition+0.5)))
            self.layers.append(nn.ReLU(True))
            curr += addition
        self.layers.append(nn.Conv1d(int(curr+0.5), 256, 51, padding=25))
        
        self.linear1 = nn.Linear(256*308,900*2)
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = x.reshape(-1, 2, 900)
        return torch.sigmoid(x)

In [11]:
## Training function
def fit(Model, device, dataloader, loss_fn, optim):
    Model.train().to(device)
    train_loss = []
    for data,label in dataloader:
        img = data
        img = img.to(device)
        label = label.to(device)
        decoded_img = Model(img)
        # print(decoded_img.shape)
        # decoded_img = decoded_img.reshape(-1,2, 308)
        loss = loss_fn(decoded_img, label)
        optim.zero_grad()
        loss.backward()
        optim.step()
        train_loss.append(loss.cpu().detach().numpy())
        # print(label, "hello")
    return np.mean(train_loss)

In [12]:
### Valid function
def val(Model, device, dataloader, loss_fn):
    Model.eval().to(device)
    with torch.no_grad(): 
        list_decoded_img = []
        list_img = []
        for  data, label in dataloader:
            img = data
            # img = img.view(img.size(0), -1).to(device) 
            # img = img[:,np.newaxis,:].to(device) 
            img = img.to(device) 
            label = label.to(device)
            decoded_img = Model(img)
            # decoded_img = decoded_img.reshape(-1,2, 308)
            list_decoded_img.append(decoded_img.cpu())
            list_img.append(label.cpu())
        list_decoded_img = torch.cat(list_decoded_img)
        list_img = torch.cat(list_img)
        # print(list_img, list_decoded_img, "vaLLLLL FUNCTION")
#         for i in range(len(list_img)):
            
        val_loss = loss_fn(list_decoded_img, list_img)
    return val_loss.data

In [ ]:
#finding global normalization numbers for NR Curves
import os
directory = "../"
totalDataY = []
totalDataX = []
count = 0
for i,filename in enumerate(os.listdir(directory)):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and f[-3:] == "npy" and f[3:9] == "NR-SLD":
        print(f[3:])
        NRCurveY = np.load(f)[:,1]
        NRCurveX = np.load(f)[:,0]
        NRCurveY = np.log10(NRCurveY)
        NRCurveX = np.log10(NRCurveX)
        for j in range(len(NRCurveX)):
            totalDataY.append(NRCurveY[j])
            totalDataX.append(NRCurveX[j])
        # totalDataY.append(NRCurveY)
        # totalDataX.append(NRCurveX)
min_valXNR = float('inf')
max_valXNR = -float('inf')
min_valYNR = float('inf')
max_valYNR = -float('inf')
totalDataY = np.array(totalDataY)
totalDataX = np.array(totalDataX)
# print(totalDataY.shape)
for i in range(len(totalDataY)):
    min_valXNR = min(min(totalDataX[i]), min_valXNR)
    min_valYNR = min(min(totalDataY[i]), min_valYNR)
    max_valXNR = max(max(totalDataX[i]), max_valXNR)
    max_valYNR = max(max(totalDataY[i]), max_valYNR)
# for i in range(len(totalDataY)):
#     for j in range(len(totalDataY[0])):
#         totalDataX[i][j] -= min_valXNR
#         totalDataY[i][j] -= min_valYNR
#         totalDataX[i][j] /= (max_valXNR - min_valXNR)
#         totalDataY[i][j] /= (max_valYNR - min_valYNR)
np.save("../NRRanges100000", [min_valXNR, max_valXNR, min_valYNR, max_valYNR])

In [ ]:
#saving SLD ranges

for first in range(1,6):
        for second in range(1,6):
            with open('SLDRanges'+str(first)+str(second)+'.npy', 'wb') as f:

                curves_SLD = np.load('../SLD_CurvesPoly'+str(first)+str(second)+'.npy')

                x_points = []
                y_points = []
                for curve in curves_SLD:
                    x_points.append(curve[0])
                    y_points.append(curve[1])
                # breaker = 0
                # # print(y_points[0])
                # for i in range(1,len(y_points[0])):
                #     if y_points[0][i]-y_points[0][i-1] != 0:
                #         breaker = i
                #         break
                # print(breaker)
                min_valX = float('inf')
                min_valY = float('inf')
                max_valX = -float('inf')
                max_valY = -float('inf')
                for i in range(len(y_points)):
                    min_valX = min(min(x_points[i]), min_valX)
                    min_valY = min(min(y_points[i]), min_valY)

                    max_valX = max(max(x_points[i]), max_valX)
                    max_valY = max(max(y_points[i]), max_valY)
                np.save(f, np.array([min_valX, max_valX, min_valY, max_valY]))
            # np.save(f, np.array([1, 3]))

In [ ]:
#Training each model

from copy import deepcopy
count = 0
for first in range(1,6):
    for second in range(1,6):
        count += 1
        with open('../SLDRanges100000.npy', 'rb') as f:
            min_valXSLD, max_valXSLD, min_valYSLD, max_valYSLD = np.load(f)
        print(min_valXSLD, max_valXSLD, min_valYSLD, max_valYSLD)
        loss_fn = torch.nn.MSELoss()
        lr= 2.15481e-05
        torch.manual_seed(0)
        Model = CNN(num_layers=12)
        params_to_optimize = Model.parameters()

        optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay= 2.6324e-05)
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        Model.to(device)
        print(first, second)
        curves_nr = np.load('../NR-SLD_CurvesPoly' + str(first) + str(second) + '.npy')
        curves_SLD = np.load('../SLD_CurvesPoly' + str(first) + str(second) + '.npy')

        curves_nr = np.log10(curves_nr)

        x_points = []
        y_points = []
        for curve in curves_nr:
            x_points.append(curve[0])
            y_points.append(curve[1])
        for i in range(len(y_points)):
            for j in range(len(y_points[0])):
                x_points[i][j] -= min_valXNR
                y_points[i][j] -= min_valYNR
                x_points[i][j] /= (max_valXNR - min_valXNR)
                y_points[i][j] /= (max_valYNR - min_valYNR)
        curves_nr2 = []
        for i in range(len(y_points)):
            curves_nr2.append([x_points[i],y_points[i]])
        #     plt.plot(x_points[i], y_points[i])
        # plt.show()
        curves_nr2 = np.stack(curves_nr2)
        curves_nr2.shape
        x_points = []
        y_points = []
        for curve in curves_SLD:
            x_points.append(curve[0].copy())
            y_points.append(curve[1].copy())
        min_valX = float('inf')
        min_valY = float('inf')
        max_valX = -float('inf')
        max_valY= -float('inf')
        for i in range(len(y_points)):
            min_valX = min(min(x_points[i]), min_valX)
            min_valY = min(min(y_points[i]), min_valY)
            max_valX = max(max(x_points[i]), max_valX)
            max_valY = max(max(y_points[i]), max_valY)
        for i in range(len(y_points)):
            for j in range(len(y_points[0])):
                x_points[i][j] -= min_valX
                y_points[i][j] -= min_valY
                x_points[i][j] /= (max_valX - min_valX)
                y_points[i][j] /= (max_valY - min_valY)
        curves_SLD2 = []
        for i in range(len(y_points)):
            curves_SLD2.append([x_points[i],y_points[i]])
        curves_SLD2 = np.stack(curves_SLD2)
        curves_SLD2.shape
        batch_size=32
        R = curves_nr[:,1]

        R_m = R[:,np.newaxis,:]
        xtrain, ytrain, xval, yval, xtest, ytest = \
        dpre.split_input_arrays(R_m,curves_SLD2, size_split=0.9)
        #Prepare data files, continuation
        train_dataset, valid_dataset, test_dataset, train_loader, valid_loader, test_loader = \
        dpre.get_dataloaders_fromsplitarrays(xtrain,ytrain,xval,yval,xtest,ytest,batch_size=batch_size)
        num_epochs=500
        #train and validate
        diz_loss = {'train_loss':[],'val_loss':[]}
        for epoch in range(num_epochs):
            train_loss = fit(Model,device,train_loader,loss_fn,optim)
            val_loss = val(Model,device,valid_loader,loss_fn)
            diz_loss['train_loss'].append(train_loss)
            diz_loss['val_loss'].append(val_loss)
            
        torch.save(Model.state_dict(), "../PolyzwitterionModels100000/ModelPoly100000" + str(first)+str(second))


0.0 509.10000000000724 -1.7698126020611056 5.396347352026074
1 1
xtrain.shape, ytrain.shape, xval.shape, yval.shape, xtest.shape, ytest.shape
(90000, 1, 308) (90000, 2, 900) (5000, 1, 308) (5000, 2, 900) (5000, 1, 308) (5000, 2, 900)


KeyboardInterrupt: 